## Question_5_Solution:

**Q-5.** Uber is a taxi service provider as we know, we need to predict the high
booking area using an Unsupervised algorithm and price for the location using a
supervised algorithm and use some map function to display the data
Dataset This is the Dataset You can use this dataset for this question.

In [1]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans  # Unsupervised algorithm or Clustering algorithm
from sklearn.ensemble import RandomForestRegressor  # Supervised algorithm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import folium # mapping library

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
# Provide the path to your CSV file in Google Drive
csv_path = '/content/drive/MyDrive/Colab Notebooks/Q5.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)
# Display the DataFrame
df.head(2)


,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,c03059f5-0d57-482f-a7b1-5b5841d3d51e,1.543275e+09,23,26,11,2018-11-26 23:33:14,America/New_York,West End,Boston University,Uber,...,0.1225,1543251600,40.45,1543233600,46.49,1543255200,37.17,1543291200,43.84,1543244400
1,50367894-71e8-48cf-a678-c76de800ebd6,1.543453e+09,0,29,11,2018-11-29 00:51:47,America/New_York,South Station,West End,Lyft,...,0.0000,1543420800,33.82,1543399200,42.72,1543438800,30.19,1543399200,36.75,1543438800


In [56]:
df.shape

(30000, 57)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           30000 non-null  object 
 1   timestamp                    30000 non-null  float64
 2   hour                         30000 non-null  int64  
 3   day                          30000 non-null  int64  
 4   month                        30000 non-null  int64  
 5   datetime                     30000 non-null  object 
 6   timezone                     30000 non-null  object 
 7   source                       30000 non-null  object 
 8   destination                  30000 non-null  object 
 9   cab_type                     30000 non-null  object 
 10  product_id                   30000 non-null  object 
 11  name                         30000 non-null  object 
 12  price                        27637 non-null  float64
 13  distance        

In [58]:
cols = df.columns
cols

Index(['id', 'timestamp', 'hour', 'day', 'month', 'datetime', 'timezone',
       'source', 'destination', 'cab_type', 'product_id', 'name', 'price',
       'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature',
       'apparentTemperature', 'short_summary', 'long_summary',
       'precipIntensity', 'precipProbability', 'humidity', 'windSpeed',
       'windGust', 'windGustTime', 'visibility', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'apparentTemperatureHigh', 'apparentTemperatureHighTime',
       'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon',
       'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex',
       'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase',
       'precipIntensityMax', 'uvIndexTime', 'temperatureMin',
       'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime',
       'apparentTemperat

In [59]:
# define numerical & categorical columns
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 46 numerical features : ['timestamp', 'hour', 'day', 'month', 'price', 'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature', 'apparentTemperature', 'precipIntensity', 'precipProbability', 'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase', 'precipIntensityMax', 'uvIndexTime', 'temperatureMin', 'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime']

We have 11 categorical features : ['id', 'datetime', 'timezone', 'source', 'destination', 'cab_type', 'product_id', 'name', 'short_summary', 'long_summary', 'icon

In [60]:
df['price'].isnull().sum()

2363

In [61]:
# Replace null values in the "price" column with the mean value
df['price'] = df['price'].fillna(df['price'].mean())

In [62]:
df['price'].isnull().sum()

0

In [63]:
df['datetime']

0        2018-11-26 23:33:14
1        2018-11-29 00:51:47
2        2018-12-03 19:07:59
3        2018-11-26 10:14:10
4        2018-12-15 14:20:15
                ...         
29995    2018-11-26 17:02:10
29996    2018-12-03 07:43:03
29997    2018-12-18 09:20:05
29998    2018-12-01 15:28:01
29999    2018-11-26 03:40:46
Name: datetime, Length: 30000, dtype: object

In [64]:
df['source'].unique()

array(['West End', 'South Station', 'Beacon Hill', 'Haymarket Square',
       'Back Bay', 'North Station', 'Northeastern University',
       'Boston University', 'Financial District', 'North End',
       'Theatre District', 'Fenway'], dtype=object)

In [65]:
df['destination'].unique()

array(['Boston University', 'West End', 'Beacon Hill', 'North End',
       'Northeastern University', 'Back Bay', 'Fenway', 'North Station',
       'South Station', 'Theatre District', 'Haymarket Square',
       'Financial District'], dtype=object)

In [66]:
# Unsupervised learning - Clustering
X = df[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
df['cluster'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [67]:
# Supervised learning - Regression
X_reg = df[['latitude', 'longitude']]
y_reg = df['price']
reg_model = RandomForestRegressor()
reg_model.fit(X_reg, y_reg)
df['predicted_price'] = reg_model.predict(X_reg)

In [69]:
# Map visualization
center_lat, center_lng = np.mean(df['latitude']), np.mean(df['longitude'])
m = folium.Map(location=[center_lat, center_lng], zoom_start=10)

for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        popup=f"Cluster: {row['cluster']}, Price: {row['predicted_price']}"
    ).add_to(m)

m.save('map.html')